In [137]:
import pandas as pd

In [138]:
df = pd.read_csv("dpschool2a.csv", delimiter=";")

In [139]:
df.head()

,title,alias,type,catid,short_description,description,geodata,latitude,longitude,geo_x,...,country,phone,url,email,additional_fields,image,icons,published,created,modified
0,WC U-Bahnhof Stiglmaierplatz,wc-bahnhof-stiglmaierplatz,toilet,188,Die behindertengerechte Toilette ist geschloss...,NaN,1,481.475.266,115.586.581,41.646.198.670.958,...,DE,089 3 56 88 08,https://www.cbf-muenchen.de/der-cbf/kontakt,info@cbf-muenchen.de,"{""wc_shortname"":""bf-stiglm1WC"",""wc_plan"":""bf-s...","{""image_small"":""images\/wc-af\/a-WC-verschwund...","{""0"":""21""}",0,3/3/2015 15:09,10/06/2021 08:10
1,WC Bad Georgenschwaige Damen,wc-bad-georgenschwaige-damen,toilet,187,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Damen-WC</strong> ist im Umklei...,1,4.817.566,1.157.174,41.621.419.913.503,...,DE,NaN,NaN,NaN,"{""wc_shortname"":""ba-george1WC1"",""wc_plan"":""ba-...","{""image_small"":""images\/wc-af\/ba-george1WC1f1...","{""0"":""4"",""1"":""11""}",1,3/3/2015 16:25,09/08/2021 09:21
2,WC Bad Georgenschwaige Herren,wc-bad-georgenschwaige-herren,toilet,187,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Herren-WC ist im Umkleidebau</s...,1,4.817.566,1.157.174,41.621.419.913.503,...,DE,NaN,NaN,NaN,"{""wc_shortname"":""ba-george1WC2"",""wc_plan"":""ba-...","{""image_small"":""images\/wc-af\/ba-george1WC2f1...","{""0"":""6"",""1"":""20""}",1,3/3/2015 16:37,09/08/2021 09:21
3,WC Freibad Unterhaching,wc-freibad-unterhaching,toilet,212,Bedingt Ã¶ffentliche Toilette mit Dusche in ei...,<p>Das rollstuhlgerechte WC des Unterhachinger...,1,480.619.761,116.208.199,41.706.291.906.759,...,Deutschland,089 6 11 40 51,NaN,NaN,"{""wc_shortname"":""ba-unterh1WC"",""wc_plan"":""ba-u...","{""image_small"":""images\/wc-af\/ba-unterh1WC1f1...","{""0"":""4"",""1"":""8"",""2"":""11""}",1,3/3/2015 17:08,10/06/2021 08:10
4,WC U-Bahnhof MÃ¼nchner Freiheit,wc-bahnhof-muenchner-freiheit,toilet,188,"Ã–ffentliche Toilette im U-Bahnhof, zugÃ¤nglic...",<p>Das rollstuhlgerechte WC des U-Bahnhofs MÃ¼...,1,481.622.924,115.870.025,41.629.998.019.931,...,DE,089 54 36 97 44,NaN,NaN,"{""wc_shortname"":""bf-muefre1WC"",""wc_plan"":""bf-m...","{""image_small"":""images\/wc-af\/bf-muefre1WC1f1...","{""0"":""4"",""1"":""11"",""2"":""12""}",1,4/3/2015 17:02,10/06/2021 08:09


In [140]:
df.drop(df[(df["latitude"] == 0)].index, inplace=True)

In [141]:
df = df[df["published"] == 1]

In [142]:
df = df[['title', 'short_description', 'description', 'latitude', 'longitude', 'address', 'addr_no', 
         'zip_code', 'city', 'country', 'additional_fields', 'image', 'icons', 'modified']]

In [143]:
import json
df["additional_fields"] = df["additional_fields"].map(lambda x: json.loads(x))
df["image"] = df["image"].map(lambda x: json.loads(x))

In [144]:
df = df[df['latitude'].notna()]

In [145]:
from pandas import isnull
df["icons"] = df["icons"].map(lambda x: {} if isnull(x) else x)
df["icons"] = df["icons"].map(lambda x: x if isinstance(x, str) else str(x))
df["icons"] = df["icons"].map(lambda x: json.loads(x))

In [146]:
df["modified"] = pd.to_datetime(df["modified"], errors="coerce")

In [147]:
df['longitude'] = df['longitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000
df['latitude'] = df['latitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000

In [148]:
df["description"] = df["description"].map(lambda x: "" if isnull(x) else x)
df["short_description"] = df["short_description"].map(lambda x: "" if isnull(x) else x)
df["addr_no"] = df["addr_no"].map(lambda x: "" if isnull(x) else x)

In [149]:
df["address"] = df["address"] + " " + df["addr_no"]

In [150]:
df.drop(["addr_no"], axis=1, inplace=True)

In [151]:
df["position"] = None
for i in df.index:
    df.at[i, 'position'] = (df.loc[i]['latitude'], df.loc[i]['longitude'])

In [152]:
icons = {"Toilette für Alle": 10, "Rolli WC": 11, "Rolli WC -DIN fern": 20, "WC eng": 17, "kein Rolli WC vorhanden": 21,
        "Parken": 7, "Stufe": 1, "WC - unisex": 4, "WC getrennt": 6, "Rollstuhl": 5, "Lift groß": 12, "Lift klein": 13,
        "Kino-Untertitel": 15, "Induktionsschleife": 14, "low-vision": 18, "kognitiv beeinträchtigt": 19, "Umkleide": 9,
        "Dusche": 8, "nicht rollstuhlgerecht": 16, "Rampe": 2, "keine Steigung - eben": 26, "moderate Steigung / Gefälle": 22,
        "weitere moderate Steigungen / Gefälle": 23, "heftige Steigung / Gefälle": 25, "weitere heftige Steigung / Gefälle": 24}
new_dict = {v: k for k, v in icons.items()}
new_dict.update({40: "Euro Key", 41: "Ramp Incline", 42: "Door Width"})

In [153]:
df["icons"] = df["icons"].map(lambda x: tuple(map(int, list(x.values()))))

In [154]:
df[40] = df["additional_fields"].map(lambda x: x["wc_euro"] if "wc_euro" in x.keys() else "0")
df[40] = df[40].map(lambda x: int(x) if x in ["1", "0"] else 0)

In [155]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

df[41] = df["additional_fields"].map(lambda x: x["wc_ramp_incline"])
df[41] = df[41].map(lambda x: x.replace(",","."))
df[41] = df[41].map(lambda x: float(x) if isfloat(x) else 0)
#need to check further as some have extra text

In [156]:
df[42] = df["additional_fields"].map(lambda x: x["wc_door_width"])
df[42] = df[42].map(lambda x: int(x) if x != "" else 150)
df[42] = df[42].map(lambda x: x if x > 15 else x*10)

In [157]:
x = pd.get_dummies(df.icons.apply(pd.Series).stack()).sum(level=0)
x = x.rename(int, axis="columns")
col = list(x.columns)
for i in [40, 41, 42]:
    col.append(i)

/tmp/ipykernel_63985/3753455811.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  x = pd.get_dummies(df.icons.apply(pd.Series).stack()).sum(level=0)


In [158]:
df = df.rename(columns={40:"eurokey", 41: "ramp_steepness", 42: "door_width"})

In [159]:
df["photo"] = df["image"].map(lambda x: x["image_small"])
df["plan"] = df["additional_fields"].map(lambda x: "images/wc-plan/" + x["wc_plan"] + ".jpg")
#df["seat_height"] = df["additional_fields"].map(lambda x: x["wc_seat"])

In [160]:
df = df.drop(df.loc[df['icons'].isin([(21,)])].index)

In [161]:
df["grip"] = df["additional_fields"].map(lambda x:x["wc_wheregrip"])

lr = flexible left & flexible right

l = flexible left

rfl & flr = fixed left and flexible right

fr = fixed right

none = no grip

lfr = flexible left and fixed right

r = flexible right

fl = fixed left

In [177]:
df[df["title"] == "WC Bahnhof Huglfing"]["additional_fields"][1039]

{'wc_shortname': 'bf-huglfi1WC1',
 'wc_plan': 'bf-huglfi1WC1',
 'wc_fotos': 'bf-huglfi1WC1f1, bf-huglfi1WC1f4',
 'wc_location': 'Vor dem Häuschen ist eine Zickzack-Rampe. Zwischen den beiden Rampenteilen und auch vor der Tür ist ein Podest. Auf einer Seite ist je eine Haltestange montiert.',
 'wc_ramp_length': '2,5 m + 3,5',
 'wc_ramp_incline': '7,5% + 6',
 'wc_infos': 'An der Tür zur behindertengerechten bzw. rollstuhlgerechten Toilette ist ein Euroschloss angebracht.<br />Das Waschbecken, gegenüber von der Tür, ist auf 67 cm unterfahrbar. Die Waschbeckenoberkante ist in 85 cm Höhe. <br />Über dem Waschbecken hängt - sehr hoch - ein Spiegel in 145 cm Höhe.',
 'wc_easy': 'Die Toilette ist gut. \r\nDer WC-Sitz ist etwas hoch.\r\nMan kann den WC-Sitz von der rechten Seite und von vorn anfahren.',
 'wc_euro': '1',
 'wc_doorknob': '100',
 'wc_doortaster': '',
 'wc_dooropens': 'ar',
 'wc_doorautopen': 'nz',
 'wc_door_width': '90',
 'wc_room_diameter': '150',
 'wc_roominfos': 'An der Tür zur

In [165]:
df["image"][1]

{'image_small': 'images/wc-af/ba-george1WC1f1.jpg',
 'image': 'images/wc-header/ba-george1WC1f2q.jpg',
 'alt': 'Bad Georgenschwaige',
 'caption': 'DamenWC Wasch- und WC-Becken.',
 'width': '100%',
 'height': '0'}

In [168]:
df["access"] = df["additional_fields"].map(lambda x:x["wc_infos"])
df["interior_description"] = df["additional_fields"].map(lambda x:x["wc_roominfos"])
df["wc_details"] = df["additional_fields"].map(lambda x:x["wc_addinfo"])
df["wc_accessleft"] = df["additional_fields"].map(lambda x:x["wc_accessleft"])
df["wc_accessright"] = df["additional_fields"].map(lambda x:x["wc_accessright"])
#df["wc_seat"] = df["additional_fields"].map(lambda x:x["wc_seat"])
#df["wc_easy"] = df["additional_fields"].map(lambda x:x["wc_easy"])

In [169]:
#df["wc_seat"] = df["wc_seat"].map(lambda x: int(x[-2:]) if x[-2:] != "" else None)

In [170]:
#df[df["wc_seat"] == ",5"]

In [171]:
df = df.rename(columns={"description": "direction"})

In [109]:
df["additional_fields"][600]["wc_easy"]

'Die Toilette ist sehr gut.\r\nEs gibt eine Pflegeliege und einen Lifter.\r\n'

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040 entries, 1 to 1091
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 1040 non-null   object        
 1   short_description     1040 non-null   object        
 2   direction             1040 non-null   object        
 3   latitude              1040 non-null   float64       
 4   longitude             1040 non-null   float64       
 5   address               1040 non-null   object        
 6   zip_code              1040 non-null   int64         
 7   city                  1040 non-null   object        
 8   country               1040 non-null   object        
 9   additional_fields     1040 non-null   object        
 10  image                 1040 non-null   object        
 11  icons                 1040 non-null   object        
 12  modified              1029 non-null   datetime64[ns]
 13  position          

In [31]:
df = df[["title", "short_description", "photo", "plan", "position", "address", "eurokey", "ramp_steepness", "door_width", "modified",
         "zip_code", "city", "grip", "direction", "access", "interior_description", "wc_details", "wc_accessright", "wc_accessleft"]]

In [32]:
errors = pd.read_excel("toilets_text errors.xlsx")

In [33]:
errors.dropna(inplace=True)

In [34]:
list(errors["Text error"])

['Ãœ', 'Ã¼', 'Ã–', 'Ã¶', 'Ã„', 'Ã¤', 'ÃŸ', 'Ã©', 'Ã¨', 'Ã‡', 'â€“']

In [35]:
import re

errors = pd.Series(errors["correction"].values,index=errors["Text error"]).to_dict()

# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in errors.items()) 
#Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))

In [36]:
df["short_description"] = df["short_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["title"] = df["title"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["address"] = df["address"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["city"] = df["city"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["direction"] = df["direction"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["access"] = df["access"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["interior_description"] = df["interior_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["wc_details"] = df["wc_details"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))

In [37]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [38]:
df["direction"] = df["direction"].map(lambda x:remove_html_tags(x))
df["access"] = df["access"].map(lambda x:remove_html_tags(x))
df["interior_description"] = df["interior_description"].map(lambda x:remove_html_tags(x))
df["wc_details"] = df["wc_details"].map(lambda x:remove_html_tags(x))

In [39]:
df["id"] = df.index

In [40]:
df.head()

,title,short_description,photo,plan,position,address,eurokey,ramp_steepness,door_width,modified,zip_code,city,grip,direction,access,interior_description,wc_details,wc_accessright,wc_accessleft,id
1,WC Bad Georgenschwaige Damen,Bedingt öffentliche Toilette in einem Freibad.,images/wc-af/ba-george1WC1f1.jpg,images/wc-plan/ba-george1WC1.jpg,"(48.1756, 11.5717)",Belgradstr. 195,0,0.0,85,2021-09-08 09:21:00,80804,München,lr,"Das Damen-WC ist im Umkleidebau, ganz hinten l...",,"Das Waschbecken ist außerhalb des WC-Raums, di...",Seitlich hinter dem WC ist auf 100 cm ein Tast...,93,85,1
2,WC Bad Georgenschwaige Herren,Bedingt öffentliche Toilette in einem Freibad.,images/wc-af/ba-george1WC2f1.jpg,images/wc-plan/ba-george1WC2.jpg,"(48.1756, 11.5717)",Belgradstr. 195,0,0.0,85,2021-09-08 09:21:00,80804,München,l,"Das Herren-WC ist im Umkleidebau, ganz hinten ...",,"Das Waschbecken ist außerhalb des WC-Raums, di...",Rechts an der Wand ist in 80 cm Höhe die Spülu...,,90,2
3,WC Freibad Unterhaching,Bedingt öffentliche Toilette mit Dusche in ein...,images/wc-af/ba-unterh1WC1f1.jpg,images/wc-plan/ba-unterh1WC.jpg,"(48.0619, 11.6208)",Schrenkstr. 2,1,0.0,75,2021-10-06 08:10:00,82008,Unterhaching,lr,Das rollstuhlgerechte WC des Unterhachinger Fr...,Das WC-Becken ist schräg links gegenüber von d...,Zwischen Umkleide und Duschbad ist eine Schieb...,,105,75,3
4,WC U-Bahnhof Münchner Freiheit,"Öffentliche Toilette im U-Bahnhof, zugänglich ...",images/wc-af/bf-muefre1WC1f1.jpg,images/wc-plan/bf-muefre1WC1.jpg,"(48.1622, 11.587)",Münchner Freiheit,1,0.0,85,2021-10-06 08:09:00,80802,München,lr,Das rollstuhlgerechte WC des U-Bahnhofs Münchn...,,Der Bügel an der Türinnenseite ist eigentlich ...,Aus der Sitzposition gesehen hängt links vom W...,100,105,4
5,WC Dynasty Unterhaching,Halböffentliche Toilette in einem Lokal.,images/wc-af/dynast1WC1f1.jpg,images/wc-plan/dynast1WC.jpg,"(48.0556, 11.6076)",Inselkammerstr. 3,0,0.0,90,2021-08-18 16:24:00,82008,Unterhaching,rfl,Auf dem Weg zum Haupteingang ist eine Schwelle...,Das WC befindet sich im Erdgeschoss des Lokals...,,,104,,5


In [ ]:
df.to_json("toilets_v14.json")

In [179]:
"new_images/background.jpeg".split(".")[0]

'new_images/background'